Entwickle eine einfache Client-Server-Anwendung
- Was ist Client-Server
- Python-Server mit Flask aufsetzen
- API-Endpunkt erstellen (JSON senden & empfangen)
- Client erstellen (python)
- Seiten-Reload verhindern & Events verstehen
- CORS verstehen (optional)
- Endprodukt zusammenführen
- OOP: Eine Klasse modellieren in UML und python
- Datenhaltung: Objekte in einer Liste verwalten

# Spickzettel 

#### Server



Imports die benötigt werden damit der Server funktionieren kann

In [ ]:
from flask import Flask, request, jsonify
from pydantic import BaseModel, Field, ValidationError
# from model import Spieler 
import json
import os
from flask_cors import CORS

#### Flask und app.route

In [ ]:
app = Flask(__name__)

# Route für die Hauptseite
@app.route('/')
def home():
    """Antwortet mit allgemeinen Anfrageinformationen."""
    response_attributes = {
        "methode": request.method,
        "args": request.args,
        "form": request.form,
        "data": request.data,
        "headers": request.headers,
        "cookies": request.cookies,
        "path": request.path,
        "url": request.url,
        "remote_addr": request.remote_addr
    }
    return jsonify(response_attributes)

# Route zum Empfangen von Nachrichten
@app.route('/message', methods=['POST'])
def handle_message():
    """Empfängt eine Nachricht, gibt diese zurück als Echo."""
    data = request.json
    message = data.get('message', '')
    return jsonify({"response": f"Echo: {message}"})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=12345)  # Startet den Server

- Mit `@app.route('/')` richtet man ein, dass der Server bei Aufruf von `127.0.0.1:Beispielport` die nachfolgende Methode aufruft. Der Rückgabewert der Methode wird an den Client gesendet.


- Mit `@app.route('/Beispiel')` richtet man eine weitere Methode ein (`127.0.0.1:12345/Beispiel`)


- Mit dem otionalen Parameter `methods`=[`P0ST`] wird festgelegt, dass die Methode zusätzlich Daten erhält. Diese liege im JSON-Format bereit.


- Die Variable `request` ist eine kontext-gebunde Variable. Sie erhält für den aktuellen Aufruf alle Daten, die durch und mit dem Aufruf entstanden sind.


- JSON ist ein Datenformat zur Übertragung zwischen Server und Client. Näheres siehe unten. Es ist - in Python - ein Dictionary.

#### pydentic und ValidationError

In [ ]:
class Spieler(BaseModel):
    name: str = Field(default = "Muster", min_lenght=2, max_lenght=20)
    alter: int = Field(default =30 ,ge=18, le=50)

# Beispiel einer Validierung:
data = {"name": "Lukas", "alter": 25}
try:
    spieler = Spieler(**data)  # Validierung erfolgt hier
    print(spieler)

    return jsonify ({
        "status": "ok",
        "message": "Spieler erfolgreich erstellt",
        "spieler": s.model_dump()
    }), 201
    
except ValidationError as e:
    return jsonify({
        "status": "error"
        "message": "Validierung fehlgeschlagen",
        "details": e.errors()
    }), 400
    print("Fehler bei der Validierung:", e)

#### JSON datei lesen und speichern

In [ ]:
dateiname = "spieler.json"

if os.path.exists(dateiname):
    with open(dateiname, "r", encoding="utf-8") as f:
        daten = json.load(f)
else:
    daten = []

# Beispiel für das Hinzufügen eines Spielers und das Speichern in der Datei:
spieler_liste = [{"name": "Lukas", "alter": 25}]
with open(dateiname, "w", encoding="utf-8") as f:
    json.dump(spieler_liste, f, ensure_ascii=False, indent=4)

## Client


In [ ]:
import requests

# Server-URL
server_url = 'http://localhost:Beispielport/message'

while True:
    # Nachricht vom Benutzer eingeben
    message = input("Nachricht an den Server: ")
    # Nachricht an den Server senden
    response = requests.post(server_url, json={"message": message})
    # Antwort vom Server anzeigen
    if response.status_code == 200:
        data = response.json()
        print(f"Antwort vom Server: {data['response']}")
    else:
        print("Fehler bei der Anfrage:", response.status_code)

In [ ]:
import requests
import json

URL = "http://localhost:Beispielport/spieler"

# 🚗 Beispiel-Daten
spieler_daten = {
    "name": str(input("Name:")),
    "jahrgang": int(input("Jahrgang(1980-2007):")),
}

response = requests.post(URL, json=spieler_daten)
print("Statuscode:", response.status_code)


if response.status_code == 201:
    print("✅ Spieler erfolgreich erstellt:")
    print(json.dumps(response.json(), indent=4, ensure_ascii=False))
else:
    print("❌ Fehler bei der Erstellung:")
    print(json.dumps(response.json(), indent=4, ensure_ascii=False))
